In [81]:
from microtc.textmodel import TextModel
import pandas as pd
from numpy import random as npr

In [109]:
train=pd.read_csv('data/train_emo.tsv',sep='\t')
dev=pd.read_csv('data/dev_emo.tsv',sep='\t') 
test=pd.read_csv('data/emoevales_test.tsv',sep='\t')

In [110]:
test.head()

,id,event,tweet,offensive
0,16b25dfb-e284-4a58-b62c-8186fc082eb6,GameOfThrones,"Señor de luz, ven a nosotros en nuestra oscuri...",NO
1,2a80f6bf-4750-4783-9bc8-fdb8ff2b94c4,SpainElection,Pues ya hemos votado tanto mi madre y yo #Elec...,NO
2,1f477a6f-3559-41ee-8ec5-2e77aee53190,WorldBookDay,"#DiaDelLibro 😇⭐❤️🙏🙌😍🌷 sentimientos, viajar con...",NO
3,838add70-748c-4635-8133-36ff0b05aeb0,WorldBookDay,"¡Feliz #DíaDelLibro📚! ”El libro es fuerza, es ...",NO
4,64e3dd59-ae5c-4b9b-bd29-987609eb95d8,SpainElection,"Pues qué queréis que os diga, este sarao polít...",NO


In [78]:
np.random.choice(tkl,10, replace=False)

<ipython-input-78-d9edd3303f85>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.random.choice(tkl,10, replace=False)


array([8, 6, -2, 1, -3, 9, 7, -1, (2, 2), (2, 1)], dtype=object)

In [113]:
df=pd.DataFrame()
df['text']=train.tweet
df['klass']=train.emotion
df1=pd.DataFrame()
df1['text']=dev.tweet
df1['klass']=dev.emotion

In [114]:
data=df.to_dict(orient='records')+df1.to_dict(orient='records')

In [115]:
len(data)

6567

In [72]:
textmodel = TextModel(weighting='entropy')
textmodel.fit(data)

In [86]:
npr.randint(len(tkl))

3

In [104]:
url_options=['none','group','delete']
weighting=['entropy','tfidf','tf']
def random_model():
    params={}
    tkn=npr.randint(len(tkl))+1
    urln=npr.randint(len(url_options))
    wn=npr.randint(len(weighting))
    token_list=npr.choice(tkl,tkn,replace=False)
    params['url_option']=url_options[urln]
    params['token_list']= token_list
    params['weighting']=weighting[wn]
    #model=TextModel(**params)
    print(params)
    return model

def random_search(n=16):
    modelos=[random_model() for i in range(n)]
    return modelos

In [105]:
modelos=random_search()


{'url_option': 'delete', 'token_list': array([4, 3, (2, 1), (3, 1)], dtype=object), 'weighting': 'entropy'}
{'url_option': 'none', 'token_list': array([7, (3, 1), (2, 2), 3, -3, 1, (2, 1)], dtype=object), 'weighting': 'entropy'}
{'url_option': 'group', 'token_list': array([2, 9, 7, 3], dtype=object), 'weighting': 'tf'}
{'url_option': 'none', 'token_list': array([9, 2, 6, -3, 3, -1, 1, (3, 1), 8, (2, 2), 7], dtype=object), 'weighting': 'tf'}
{'url_option': 'group', 'token_list': array([(2, 2), -2], dtype=object), 'weighting': 'tfidf'}
{'url_option': 'delete', 'token_list': array([(2, 2), -1, 1, 4, 9, (2, 1), 6, 7, -2], dtype=object), 'weighting': 'entropy'}
{'url_option': 'delete', 'token_list': array([6, 1, (2, 1), -3, 8, 2, 9, -2, 7, 3, (3, 1), 4], dtype=object), 'weighting': 'tf'}
{'url_option': 'none', 'token_list': array([4, (3, 1), 6, (2, 1), 2, 7, -3, 8, -1, -2, 3, 1, 9], dtype=object), 'weighting': 'tfidf'}
{'url_option': 'group', 'token_list': array([9, 2, 1, (2, 1), 4], dtype=

<ipython-input-104-487a56a71caa>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  token_list=npr.choice(tkl,tkn,replace=False)


In [116]:
modelo=modelos[-1]
modelo.fit(data)

In [125]:
X=modelo.transform(data)
Xt=modelo.transform(test.tweet)

In [119]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

In [124]:
l=LabelEncoder()
labels=[e['klass'] for e in data]
l.fit(labels)
y=l.transform(labels)

array([4, 4, 5, ..., 5, 4, 3])

In [127]:
clf=LinearSVC()
clf.fit(X,y)

LinearSVC()

In [128]:
yp=clf.predict(Xt)

In [132]:
df3=pd.DataFrame()
df3['id']=test['id']
df3['yp']=l.inverse_transform(yp)

In [135]:
df3.to_csv('test_try01.tsv', header=None, index=None,sep='\t')

In [136]:
!head test_try01.tsv

16b25dfb-e284-4a58-b62c-8186fc082eb6	others
2a80f6bf-4750-4783-9bc8-fdb8ff2b94c4	others
1f477a6f-3559-41ee-8ec5-2e77aee53190	joy
838add70-748c-4635-8133-36ff0b05aeb0	joy
64e3dd59-ae5c-4b9b-bd29-987609eb95d8	joy
7bc2b439-817a-4a22-9aa3-6ac9e8ebee90	others
f785d8c5-575a-47e1-9eb4-4217bc8b6bf5	others
e0c36651-df9a-4e3a-9763-84f9a8e3c9b4	others
0d1ddeb6-ceeb-44b2-9aaf-10fa733e31cb	others
53de65ff-4f75-4118-b1cb-967647a79e65	others


## microTC TextModel token_list

## $n$-gramas 
Indica que se generen todas las subsecuencias de $n$ palabras (es decir, $m - n + 1$) tokens para un texto con $m$ palabras). se indica mediante valores negativos. 

In [2]:
texto="~buenos dias a todos ustedes~"
textmodel = TextModel(token_list=[-2], weighting ='entropy')
textmodel.compute_tokens(texto)

[['buenos~dias', 'dias~a', 'a~todos', 'todos~ustedes']]

## $q$-gramas

son $n$-gramas a nivel de carácter, es decir, cada token es una subcadena de tamaño $q$.

In [3]:
textmodel = TextModel(token_list=[5])
texto="~buenos dias a todos ustedes~"
textmodel.compute_tokens(texto)


[['q:~buen',
  'q:bueno',
  'q:uenos',
  'q:enos ',
  'q:nos d',
  'q:os di',
  'q:s dia',
  'q: dias',
  'q:dias ',
  'q:ias a',
  'q:as a ',
  'q:s a t',
  'q: a to',
  'q:a tod',
  'q: todo',
  'q:todos',
  'q:odos ',
  'q:dos u',
  'q:os us',
  'q:s ust',
  'q: uste',
  'q:usted',
  'q:stede',
  'q:tedes',
  'q:edes~']]

## skip-gramas
son $n$-gramas que saltan partes intermedias en subsecuencias. Para este caso, debe especificar la longitud de la subsecuencia y el número de palabras intermedias que se deben omitir.

In [33]:
textmodel = TextModel(token_list=[])
textmodel.compute_tokens(texto)

[[]]

In [5]:
texto="~buenos dias a todos ustedes por ahora jjjj~"

In [6]:
textmodel.fit([texto])

In [7]:
textmodel.model._w2id

{'buenos~todos~ahora': 0, 'dias~ustedes~j': 1}

In [8]:
textmodel.model['buenos']

[]

In [73]:
ng=[-1,-2,-3,]
qg=[1,2,3,4,6,7,8,9]
sg=[(3,1),(2,2),(2,1)]
tkl=ng+qg+sg

In [37]:
from itertools import combinations
def conjunto_potencia(lista):
    cp=[]
    for i in range(0,len(lista)+1):
        for comb in combinations(lista,i):
            cp.append(comb)
    return cp

In [13]:
combinaciones={}
i=0
for n in conjunto_potencia(ng)[1:]:
    for q in conjunto_potencia(qg):
        for s in conjunto_potencia(sg):
            n=str(bin(i))[2:]
            clave=n.zfill(13)
            combinaciones[clave]=(list(n)+list(q)+list(s))
            i=i+1

In [34]:
[d for d in list(combinaciones.keys())[-1]]

['1', '1', '0', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']

In [39]:
combinaciones['0000000001001']

['1', '0', '0', '1', 1, (3, 1)]

In [40]:
conjunto_potencia(ng)

[(), (-1,), (-2,), (-3,), (-1, -2), (-1, -3), (-2, -3), (-1, -2, -3)]

In [146]:
bin(128)[2:]

'10000000'

In [166]:
3**6*2**4*14336*4**2*2**3*3

64210599936